In [1]:
import os
import yaml
import pandas as pd

from utility import set_random_seed
from data import load_symbol_dfs
from combination import AlphaCombinationModel
from tokenizer import AlphaTokenizer
from alpha_generation_env import AlphaGenerationEnv
from generator import RLAlphaGenerator
import pathlib
from glob import glob

config_path = "configs/trial_config.yaml"

with open(config_path, "r") as f:
    cfg = yaml.safe_load(f)

set_random_seed(cfg.get("random_seed", 42))

gen_cfg = cfg["generator"]
data_cfg = cfg["data"]


In [2]:
# 加载数据

if True:  # 拆包
    n = data_cfg["n"]

    symbol_dict = data_cfg["symbol"]
    path = data_cfg["path"]

    start_date, end_date = data_cfg["date_range"][0], data_cfg["date_range"][1]

data_dfs = load_symbol_dfs(
    directory=path, symbols=symbol_dict, start_date=start_date, end_date=end_date, n=n
)

data = pd.concat(data_dfs["rb"])

In [3]:
# 模块初始化
combo = AlphaCombinationModel(max_pool_size=cfg["model"]["max_pool_size"])
combo.inject_data(data, target_col=data_cfg["target_col"])

tokenizer = AlphaTokenizer()
env = AlphaGenerationEnv(
    combo_model=combo, tokenizer=tokenizer, max_len=cfg["env"]["max_len"]
)

gen_cfg["vocab_size"] = tokenizer.vocab_size
gen_cfg["max_seq_len"] = cfg["env"]["max_len"]
agent = RLAlphaGenerator(env=env, config=gen_cfg)

print(f"Starting training for {gen_cfg['num_iterations']} iterations...")
agent.train(num_iterations=gen_cfg["num_iterations"])

out_cfg = cfg["output"]
os.makedirs(os.path.dirname(out_cfg["alphas_weights_path"]), exist_ok=True)
results = pd.DataFrame(
    {"expr": combo.expr_list, "ic": combo.ic_list, "weight": combo.weights}
)
results.to_csv(out_cfg["alphas_weights_path"], index=False)
print(f"Saved discovered alphas and weights to {out_cfg['alphas_weights_path']}")

Starting training for 300 iterations...
[Iter 0010]  AvgReturn=+0.0007   ComboIC=-0.0758
[Iter 0020]  AvgReturn=+0.0320   ComboIC=-0.0172
[Iter 0030]  AvgReturn=+0.0102   ComboIC=-0.1649


/Users/sumen/Documents/workspace/25summer/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/sumen/Documents/workspace/25summer/.venv/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


[Iter 0040]  AvgReturn=+0.0112   ComboIC=-0.0094
[Iter 0050]  AvgReturn=-0.0033   ComboIC=-0.0741
[Iter 0060]  AvgReturn=-0.0408   ComboIC=-0.0488
[Iter 0070]  AvgReturn=-0.0013   ComboIC=-0.2521
[Iter 0080]  AvgReturn=+0.0018   ComboIC=-0.2521
[Iter 0090]  AvgReturn=+0.0467   ComboIC=+0.0289
[Iter 0100]  AvgReturn=+0.0457   ComboIC=+0.0157
[Iter 0110]  AvgReturn=+0.0101   ComboIC=+0.0001
[Iter 0120]  AvgReturn=+0.0029   ComboIC=+0.0029
[Iter 0130]  AvgReturn=-0.0037   ComboIC=-0.0027
[Iter 0140]  AvgReturn=-0.0308   ComboIC=+0.3454
[Iter 0150]  AvgReturn=+0.0018   ComboIC=+0.3454
[Iter 0160]  AvgReturn=+0.0023   ComboIC=+0.3454
[Iter 0170]  AvgReturn=+0.0002   ComboIC=+0.3454
[Iter 0180]  AvgReturn=+0.0014   ComboIC=+0.3454
[Iter 0190]  AvgReturn=-0.0470   ComboIC=+0.2779
[Iter 0200]  AvgReturn=+0.0005   ComboIC=-0.0029
[Iter 0210]  AvgReturn=-0.0033   ComboIC=-0.0029
[Iter 0220]  AvgReturn=-0.0321   ComboIC=-0.0073
[Iter 0230]  AvgReturn=+0.0009   ComboIC=-0.0073
[Iter 0240]  AvgRetu